In [424]:
import json

# Used to display points and shapes on a map
from ipyleaflet import GeoJSON, Map

# Main package
from shape2geosparql import convert

# Used to run geospatial queries on Fuseki
from SPARQLWrapper import GET, JSON, POST, SPARQLWrapper

### Converting the shapefile to triples

In [425]:
# Convert a shapefile
converter = convert('data/trento_stations/stazioni.shp')

In [427]:
# How do the triples look like?

print(converter.write().decode())

<http://www.example.org/shape2geosparql/stazioni/data/0_geom> <http://www.opengis.net/ont/geosparql#asWKT> "POINT (11.120056 46.0726)"^^<http://www.opengis.net/ont/geosparql#wktLiteral> .
<http://www.example.org/shape2geosparql/stazioni/data/9> <http://www.example.org/shape2geosparql/stazioni/ontology/tratta> "VALSUGANA" .
<http://www.example.org/shape2geosparql/stazioni/data/6_geom> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.opengis.net/ont/sf#Geometry> .
<http://www.example.org/shape2geosparql/stazioni/data/5> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.opengis.net/ont/sf#Feature> .
<http://www.example.org/shape2geosparql/stazioni/data/0> <http://www.example.org/shape2geosparql/stazioni/ontology/nome> "FS - Trento" .
<http://www.example.org/shape2geosparql/stazioni/data/9_geom> <http://www.opengis.net/ont/geosparql#asWKT> "POINT (11.142646 46.06487)"^^<http://www.opengis.net/ont/geosparql#wktLiteral> .
<http://www.example.org/shape2geosparql/stazi

In [428]:
# A different serialization format can be used ('nt' being the default)

print(converter.write(outformat='turtle').decode())

@prefix ns1: <http://www.opengis.net/ont/geosparql#> .
@prefix ns2: <http://www.example.org/shape2geosparql/stazioni/ontology/> .
@prefix ns3: <http://github.com/nvitucci/shape2geosparql/ontology#> .
@prefix ns4: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.example.org/shape2geosparql/stazioni/data/0> a <http://www.opengis.net/ont/sf#Feature> ;
    ns2:nome "FS - Trento" ;
    ns2:tratta "FFSS" ;
    ns1:hasGeometry <http://www.example.org/shape2geosparql/stazioni/data/0_geom> .

<http://www.example.org/shape2geosparql/stazioni/data/1> a <http://www.opengis.net/ont/sf#Feature> ;
    ns2:nome "FTM - Trento" ;
    ns2:tratta "FTM" ;
    ns1:hasGeometry <http://www.example.org/shape2geosparql/stazioni/data/1_geom> .

<http://www.example.org/shape2geosparql/s

In [429]:
# Get the actual graph to run some SPARQL queries

graph = converter.graph

# The GeoSPARQL ontology does not define a property to serialize geometries as GeoJSON,
# so we use the one defined in shape2geosparql; we also use the automatically-generated
# ontology that describes the features
geojson = graph.query('''
    PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    PREFIX s2g: <http://github.com/nvitucci/shape2geosparql/ontology#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?feature ?geojson
    WHERE {
        ?feature ont:nome "FS - Trento"; 
                 geosparql:hasGeometry ?geom .
        ?geom s2g:asGeoJson ?geojson
    }
''')

print(geojson.bindings)

[{rdflib.term.Variable('feature'): rdflib.term.URIRef('http://www.example.org/shape2geosparql/stazioni/data/0'), rdflib.term.Variable('geojson'): rdflib.term.Literal('{ "type": "Point", "coordinates": [ 11.120056, 46.0726 ] }')}]


In [430]:
# Get the GeoJSON content
geojson_value = str(geojson.bindings[0]['geojson'])

print(geojson_value)

{ "type": "Point", "coordinates": [ 11.120056, 46.0726 ] }


### Displaying the points on a map

In [431]:
# Create a map and center it on Trento, Italy
m = Map(center=(46.07, 11.15), zoom=13)

In [432]:
# Create a GeoJSON layer from our GeoJSON string
geojson_layer = GeoJSON(data=json.loads(geojson_value))

# Add the layer to the map
m.add_layer(geojson_layer)

In [433]:
# Display the map
m

Map(center=[46.07, 11.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [434]:
# Add all the points instead of just one

geojson = graph.query('''
    PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    PREFIX s2g: <http://github.com/nvitucci/shape2geosparql/ontology#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?geom ?geojson
    WHERE {
        ?geom s2g:asGeoJson ?geojson
    }
''')

geojson_values = map(lambda el: json.loads(str(el['geojson'])), geojson.bindings)

In [435]:
# Display all the points on the map, one point per layer

m = Map(center=(46.07, 11.15), zoom=12)

for geojson_value in geojson_values:
    geo_json = GeoJSON(data=geojson_value)
    m.add_layer(geo_json)

m

Map(center=[46.07, 11.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Displaying polygons on a map

In [445]:
converter = convert('data/trento_parking/zone_parcheggio.shp')
graph = converter.graph

geojson = graph.query('''
    PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
    PREFIX s2g: <http://github.com/nvitucci/shape2geosparql/ontology#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?geom ?geojson
    WHERE {
        ?geom s2g:asGeoJson ?geojson
    }
''')

geojson_values = list(map(lambda el: json.loads(str(el['geojson'])), geojson.bindings))

In [446]:
# Example of polygon in GeoJSON
print(geojson_values[0])

{'type': 'Polygon', 'coordinates': [[[11.127931787965835, 46.06733702247199], [11.128328951680194, 46.067784264236366], [11.12854825979971, 46.0677095077211], [11.12898383662842, 46.06753156149367], [11.129132833584624, 46.067371278755076], [11.129431459362271, 46.06723520708073], [11.12919767140206, 46.06695605617613], [11.129248951722646, 46.06686059700935], [11.129145799276966, 46.06607048978329], [11.128504534895333, 46.06490340622377], [11.128115077563661, 46.0648206605723], [11.128026417197448, 46.06466809877524], [11.12787325832586, 46.06442582203101], [11.127258821859375, 46.064535101708685], [11.125854118153557, 46.06478176372753], [11.1247314083356, 46.064964660800406], [11.124646339050859, 46.0648537395411], [11.123329594725126, 46.06495022915264], [11.123336697033503, 46.06488259483858], [11.123276919036359, 46.064883702542886], [11.123255646830659, 46.06496060419686], [11.123227721080418, 46.06503312576825], [11.123220508394837, 46.06505576061038], [11.123126052878657, 46.

In [447]:
m = Map(center=(46.07, 11.15), zoom=13)

# Define a style for the polygons
style = {'color': 'blue', 'weight': 1.5, 'dashArray': '5', 'fillOpacity': 0.3}

for geojson_value in geojson_values:
    geo_json = GeoJSON(data=geojson_value, style=style)
    m.add_layer(geo_json)

m

Map(center=[46.07, 11.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

### Running geospatial queries

In order to run the queries in the following section, Fuseki GeoSPARQL server must be running (see the README file)

In [448]:
# Converting the first dataset again, for convenience
converter = convert('data/trento_stations/stazioni.shp')

In [449]:
# Both the query endpoint and the update endpoint are needed, since the
# GeoSPARQL dataset has to be loaded on the server
endpoint = SPARQLWrapper('http://localhost:3030/ds/sparql', updateEndpoint='http://localhost:3030/ds/update')

In [450]:
# Insert the data with a standard "INSERT DATA" query; to do this, the data have to be:
# - serialized in the 'nt' (N-Triples) format;
# - reasonably small.
# If the data are large, either insert them in batches or serialize them in a file to be uploaded.
endpoint.setMethod(POST)
endpoint.setQuery('INSERT DATA {%s}' % converter.write(outformat='nt').decode())

# Nothing needs to be returned
endpoint.query()

In [451]:
endpoint.setMethod(GET)
endpoint.setReturnFormat(JSON)

# Besides the custom and the GeoSPARQL namespaces, two more namespaces are used:
# - the "units" namespace, which defines the units (e.g. kilometers, meters, miles, etc.)
# for the geospatial queries (see https://jena.apache.org/documentation/geosparql/index#units-uri)
# - the Apache Jena geospatial filter functions namespace, to be used in FILTERs
# (see https://jena.apache.org/documentation/geosparql/index#filter-functions)
#
# Other namespaces are available for more operations 
# (https://jena.apache.org/documentation/geosparql/index#apache-jena-spatial-functionswgs84-geo-predicates).
#
# In this query, we look for features within 1 km from the specified point having
# latitude = 46.054385 and longitude = 11.13544.

endpoint.setQuery('''
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX units: <http://www.opengis.net/def/uom/OGC/1.0/>
    PREFIX spatialF: <http://jena.apache.org/function/spatial#>
    PREFIX ont: <http://www.example.org/shape2geosparql/stazioni/ontology/>
    
    SELECT ?feature ?name ?geom 
    WHERE {
        ?feature ont:nome ?name;
                 geo:hasGeometry/geo:asWKT ?geom .
        BIND (spatialF:convertLatLon(46.054385, 11.13544) AS ?point)
        FILTER (spatialF:nearby(?geom, ?point, 1, units:kilometer))}
''')

res = endpoint.query()

In [452]:
# See the results in JSON format
print(json.dumps(res.convert(), indent=2))

{
  "head": {
    "vars": [
      "feature",
      "name",
      "geom"
    ]
  },
  "results": {
    "bindings": [
      {
        "feature": {
          "type": "uri",
          "value": "http://www.example.org/shape2geosparql/stazioni/data/6"
        },
        "name": {
          "type": "literal",
          "value": "VALS - S. Chiara"
        },
        "geom": {
          "type": "literal",
          "datatype": "http://www.opengis.net/ont/geosparql#wktLiteral",
          "value": "POINT (11.13544 46.054385)"
        }
      },
      {
        "feature": {
          "type": "uri",
          "value": "http://www.example.org/shape2geosparql/stazioni/data/7"
        },
        "name": {
          "type": "literal",
          "value": "VALS - S. Bartolomeo"
        },
        "geom": {
          "type": "literal",
          "datatype": "http://www.opengis.net/ont/geosparql#wktLiteral",
          "value": "POINT (11.135147 46.04746)"
        }
      }
    ]
  }
}
